In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.TwoDoF as TwoDof

from matplotlib.lines import Line2D
import multiprocessing as mp

me.mechanics_printing()

In [9]:
def Model_Eigen(fixed_fold = None):   
    dataset_name = 'Test'
    # Set JEC Specific parameters
    sm = mf.SymbolicModel.from_file('2DoF_Model.py')
    p = TwoDof.base_params()
    
    p.rho.value = 0.0885
    p.s.value = 5.8
    p.c_root.value = 3.71
    p.c_tip.value = 2.137
    p.m.value = 943*0.66
    p.m_w.value = 2040
    p.f_0.value = 2
    p.Lambda.value = np.deg2rad(25) 
    
    p.k_w.value = (p.f_0.value*2*np.pi)**2*(p.m_w.value+p.m.value)*0.9             # Stiffness of the inner wing
    p.I_xx.value = 1/12*p.m.value*p.s.value**2          # FWT polar moment of inetia
    p.I_xx.value = 6.5/81*p.m.value*p.s.value**2 
    p.l_com.value = p.s.value/3
   
    vars_ls =[]
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    vars_ls.append((p.V,np.linspace(0,900,901))) # V must be second
    vars_ls.append((p.alpha_r,np.deg2rad([0,5,10])))
    vars_ls.append((p.c_d_max,[0,1]))

    variables = [k for k,v in vars_ls]
    
    calc_fixed = True
    if fixed_fold == None:
        flutdf = rw.eigen_perm_params(p,sm,vars_ls,calc_fixed_points=True,fp = np.array([0,np.deg2rad(-30)]))
        flutdf.to_pickle(f'Test_FWT_free.pkl')
    else:
        flutdf = rw.eigen_perm_params(p,sm,vars_ls,calc_fixed_points=False,fp = np.array([0,np.deg2rad(fixed_fold)]))
        flutdf.to_pickle(f'Test_FWT_Locked_{fixed_fold}_degrees.pkl')

In [10]:
pool = mp.Pool(mp.cpu_count())

for v in [None,-15,-7.5,0,7.5,15]:
    pool.apply_async(Model_Eigen,args=(v,))
pool.close()
pool.join()

/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
